# Ejercicos word Capitulo 3
- A. Realizar todos los ejercicios propuestos de libro (HECHO En notebook Capitulo_3_ejs
- B. Leer el CSV del ejemplo del cap2 y obtener la estructura del schema dado por defecto.
- C. Cuando se define un schema al definir un campo por ejemplo StructField('Delay', FloatType(), True) ¿qué significa el último parámetro Boolean?
- D. Dataset vs DataFrame (Scala). ¿En qué se diferencian a nivel de código?
- E. Utilizando el mismo ejemplo utilizado en el capítulo para guardar en parquet y guardar los datos en los formatos: JSON, CSV (dándole otro nombre para evitar sobrescribir el fichero origen), AVRO

Como siempre, iniciamos la SparkSession primero:

In [1]:
import pyspark
from pyspark.sql import SparkSession 

# Creamos SparkSession (IMPORTANTE cambiar el nombre si estamos en otros scripts)
 # Añadimos .config con el paquete para poder usar archivos AVRO
spark = (SparkSession
         .builder
         .appName("Ejs_word_cap3")
         .config('spark.jars.packages', 'org.apache.spark:spark-avro_2.12:3.0.1')  # añadimos libreria para ficheros AVRO
         .getOrCreate())

### B. Leer el CSV del ejemplo del cap2 y obtener la estructura del schema dado por defecto.

In [2]:
from pyspark.sql.types import *
from pyspark.sql.functions import *
 
## Cargamos el DF del capitulo 2, el de m&m
# Primero, definimos su schema Programaticamente
schema_mnm_df =  StructType([StructField('State', StringType(), True),
                             StructField('Color', StringType(), True),
                             StructField('Count', IntegerType(), True)
                            ])

# O Definimos su schema usando DDL
schema_mnm_df2 = "State STRING, Color STRING, Coutn INT"

# Lo cargamos con spark.read.csv indicando primero ruta, cabecera y el schema
mnm_df = spark.read.csv("./Datasets/sf-fire-calls.csv", header=True, schema=schema_mnm_df).show(2)

+--------+-----+-------+
|   State|Color|  Count|
+--------+-----+-------+
|20110016|  T13|2003235|
|20110022|  M17|2003241|
+--------+-----+-------+
only showing top 2 rows



In [3]:
# Tambien podemos cargar de csv infiriendo su schema
# Cargamos DataFrame de M&M
mnm_df_infer = ( spark.read.format("csv") 
                           .option("header", "true")
                           .option("inferSchema", "true")
                           .load("./Datasets/mnm_dataset.csv") )

# Vemos el schema definido en DDL
print(mnm_df_infer.printSchema())

# Puedes ver el schema creado en DDL de manera programatica invocando el shcema
mnm_df_infer.schema

root
 |-- State: string (nullable = true)
 |-- Color: string (nullable = true)
 |-- Count: integer (nullable = true)

None


StructType(List(StructField(State,StringType,true),StructField(Color,StringType,true),StructField(Count,IntegerType,true)))

### C. Cuando se define un schema al definir un campo por ejemplo StructField('Delay', FloatType(), True) ¿qué significa el último parámetro Boolean?

StructField('Delay', BooleanType(), True) )

El data type booleano representa datos binarios (V/F, 0/1)

Lista de pyspark.sql.types: 
- DataType
- NullType
- StringType
- BinaryType
- BooleanType
- DateType
- TimestampType
- DecimalType
- DoubleType
- FloatType
- ByteType
- IntegerType
- LongType
- ShortType
- ArrayType
- MapType
- StructField
- StructType

In [5]:
# Ejemplo de schema con datos boolean
boolean_schema = StructType([ StructField('Boolean', BooleanType(), True) ])

### D. Dataset vs DataFrame (Scala). ¿En qué se diferencian a nivel de código?
Un Dataset es una colección fuertemente tipificada de objetos específicos del dominio que pueden ser transformados en paralelo
utilizando operaciones funcionales o relacionales. 

Tenemos que tener en cuenta que en scala/java un DataFRame = Dataset<Row> (es decir un Dataset cuyo tipo de dato es generico, el Row).

En los lenguajes soportados por Spark (R,Python,Scala y Java), los Datasets sólo tienen sentido en Java y Scala, mientras que en Python y R sólo tienen sentido los DataFrames. Esto se debe a que Python y R no son compilados; los tipos se infieren o asignan dinámicamente durante la ejecución, no durante el tiempo de compilación.

Spark2.0 unifico la API de DF y Dataset como una API estrucutrada con interfaces similares para que desarolladores no tuvieran que aprender una de las dos APIS. El Dataset API tiene una serie de ventajas como la detección de errores en tiempo de compilación entre otros que SOLO pueden ser aprovechadas por Scala, por lo que como estamos utilizando Python, seguiremos usando DF.

Recapitulando, las operaciones que podemos realizar en Datasets en Scala/Java como -filter(), map(), groupBy(), select(), take(), etc. - son similares a las de los DataFrames. En cierto modo, los Datasets son similares a los RDD en el sentido de que proporcionan una interfaz similar a sus métodos mencionados y la seguridad en tiempo de compilación, pero con una lectura mucho más fácil y una de programación orientada a objetos.


### E. Utilizando el mismo ejemplo utilizado en el capítulo para guardar en parquet y guardar los datos en los formatos: JSON, CSV (dándole otro nombre para evitar sobrescribir el fichero origen), AVRO

Lo haremos con el DF mnm_df. 
Todos los ejemplos están en el directorio "./Datasets/DFs_saved".

Destacamos que se crean X particiones por defecto, dividiendose el archivo en Xs archivos, podemos modificar esta forma de particionar con metodos como repartition y coalesce, por lo que reduciremos este numero de particiones a 1 para este DF.

In [6]:
# Cargamos DataFrame de M&M
mnm_df = ( spark.read.format("csv") 
                           .option("header", "true")
                           .option("inferSchema", "true")
                           .load("./Datasets/mnm_dataset.csv").repartition(1) ) 
# añadimos repartition para que los guarde en 1 archivo

#### E.1 Formato CSV 
https://sparkbyexamples.com/pyspark/pyspark-read-csv-file-into-dataframe/#write-csv-file

In [7]:
from pyspark.sql.functions import *
# Guardar en CSV
( mnm_df.write.mode('overwrite') 
             .options(header='True', delimiter=',') 
             .csv("./Datasets/DFs_saved/mnm_df_csv") )

#### E.2 Formato JSON
https://sparkbyexamples.com/spark/spark-read-and-write-json-file/#write-json-file

In [7]:
# Guardar en JSON ()
mnm_df.write.mode('overwrite') \
             .json("./Datasets/DFs_saved/mnm_df_json") 

#### E.3 Formato Parquet
https://sparkbyexamples.com/pyspark/pyspark-read-and-write-parquet-file/

El formato columnar parquet es el formato por defecto en Spark. Es la mas soportada por diferentes plataformas y frameworks, open source y la hacen la más optimizada y eficiente, sobre todo cuando el DF contiene muchas columnas. Es recomendable guardar los DFs limpios y transformados en este formato.

Parquet es guardado en un directorio que contiene la estructura de los datos, los metadatos y un numero de archivos (dependiendo del numero de particiones que tenga spark para procesar).

**Es muy importante destacar que para este tipo de archivos parquet NO es necesario especificar un schema por que está incluido en los metadatos**

In [8]:
# Guardar DF como parquet
( mnm_df.write.mode('overwrite') 
            .parquet("./Datasets/DFs_saved/mnm_df_parquet") )

#### E.4 Formato AVRO
https://sparkbyexamples.com/spark/read-write-avro-file-spark-dataframe/

Destacamos que para usar archivos de formato AVRO hay que iniciar una librería externa, inciandola en la parte .config de nuestra SparkSession (mirar arrriba)

In [8]:
from pyspark.sql.avro.functions import *

# Guardar DF como AVRO
( mnm_df.write.mode('overwrite') 
             .format("avro") 
             .save("./Datasets/DFs_saved/mnm_df_avro") )

### F. Investigar sobre metodos repartition y coalesce para guardar archivos en diferentes formatos
https://sparkbyexamples.com/spark/spark-repartition-vs-coalesce/

Spark divide los datos en varias particiones, ya sea a la hora de guardar los archivos en diferentes formatos (como vimos en el ejercicio anterior de guardar en CSV, que lo guarda en 10 particiones diferentes) o de leer los datos, ejecutando las operaciones en particiones de datos de forma paralela, permitiendo trabajar más rapido.

A veces, será necesario aumentar o disminuir las particiones en función de la distribución de los datos, para ello usamos los metosos coalesce() y repartition().

En el ejercicio anterior, Spark particionó los datos de manera predeterminada, vamos a cambiar con los metodos mencionados esto para poder obtener menos archivos a la hora de guardar/leer.

Metodos: 
- getNumPartitions() muestra el numero de particiones por defecto
- coalesce() disminuye el número de particiones de un DF o RDD de manera eficiente
- repartition() aumenta o disminute el numero de particiones de un DF o RDD

A destacar la doc de spark indica que repartition y coalesce son operaciones muy costosas, por lo que se debe intentar minimizar su uso tanto como sea posible. Vamos a guardar los archivos solo en un formato por cada metodo puesto que se crean archivos bastante pesados.

##### rdd.getNumPartitions() muestra el numero de particiones por defecto

Observamos que el numero de particiones por defecto que tiene el DF original con el que hicimos el ejercicio anterior es de 12

In [17]:
# DF original
mnm_df.rdd.getNumPartitions()

1

##### repartition() aumenta o disminute el numero de particiones de un DF o RDD

In [18]:
# Usamos repartition para 
mnm_df_partition_1 = mnm_df.repartition(1)

# Comprobamos que hemos configurado las particiones a 1
mnm_df_partition_1.rdd.getNumPartitions()

# Lo guardamos en formato csv y comprobamos en el explorador que se guardó en 1 archivo csv solo
mnm_df_partition_1.write.mode('overwrite') \
                     .options(header='True', delimiter=',') \
                     .csv("./Datasets/DFs_saved/mnm_df_csv_partition1")

#####  coalesce() disminuye el número de particiones de un DF o RDD de manera eficiente

In [20]:
# Usamos repartition para 
mnm_df_coalesce_1 = mnm_df.coalesce(1)

# Comprobamos que hemos configurado las particiones a 1
mnm_df_partition_1.rdd.getNumPartitions()

# Lo guardamos en formato parquet y comprobamos en el explorador que se guardó en 1 archivo parquet solo
mnm_df_coalesce_1.write.mode('overwrite') \
                     .options(header='True', delimiter=',') \
                     .parquet("./Datasets/DFs_saved/mnm_df_parquet_coalesce")

In [21]:
spark.stop()